In [ ]:
from sagemaker.chainer import Chainer
from sagemaker import get_execution_role

role = get_execution_role()

bucket_name ="<bucketname>"

sagemaker_session = sagemaker.Session()

This launches the job on a device with no cpu.. hence slow ..

In [ ]:
chainer_estimator = Chainer("train.py",
                            role=role,
                        train_instance_type="ml.p2.large",
                        train_instance_count=1,
                        source_dir="custom_chainer",
                        hyperparameters = {'epoch': 20, 'batchsize': 64, 'traindata':'sample.csv'})
chainer_estimator.fit({'train': 's3://{}/sample.csv'.format(bucket_name),
                   'test': 's3://{}/test.csv'.format(bucket_name)})

In [ ]:
import boto3
import tarfile
from urllib.parse import urlparse
import os

def retrieve_output_from_s3(s3_url, output_dir):
    """
    Downloads output artifacts from s3 and extracts them into the given directory.
    Args:
        s3_url: S3 URL to the output artifacts
        output_dir: directory to write artifacts to
    """
    o = urlparse(s3_url)
    s3 = boto3.resource('s3')
    output_data_path = os.path.join(output_dir)
    output_file_name = os.path.join(output_data_path, 'output.tar.gz')
    try:
        os.makedirs(output_data_path)
    except FileExistsError:
        pass
    s3.Bucket(o.netloc).download_file(o.path.lstrip('/'), output_file_name)
    tar = tarfile.open(output_file_name)
    tar.extractall(output_data_path)
tar.close()

In [ ]:


chainer_training_job = chainer_estimator.latest_training_job.name

desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=chainer_training_job)
output_data = desc['ModelArtifacts']['S3ModelArtifacts'].replace('model.tar.gz', 'output.tar.gz')

#retrieve_output_from_s3(output_data, 'output/sentiment')

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')